In [48]:
import numpy as np
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from datetime import datetime
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.compose import ColumnTransformer
import sklearn
from sklearn.neural_network import MLPRegressor


In [49]:
train = pd.read_csv("cleaned_train.csv")
test = pd.read_csv("cleaned_test.csv")
ids = test["id"]

In [50]:
def train_multinomial_logistic_regression(data, target_column, test_size=0.2, random_state=42):
    """
    Trains a multinomial logistic regression classifier and evaluates its performance.

    Parameters:
    - X: Features (independent variables)
    - y: Target variable (dependent variable)
    - test_size: Proportion of the dataset to include in the test split (default is 0.2)
    - random_state: Seed used by the random number generator (default is 42)

    Returns:
    - classifier: Trained multinomial logistic regression classifier
    - accuracy: Accuracy of the classifier on the test set
    - conf_matrix: Confusion matrix
    - class_report: Classification report
    """

    X = data.drop([target_column, 'id'], axis=1)
    y = data[target_column]
    y = y.astype(np.uint8)

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Create a multinomial logistic regression classifier
    # classifier = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000) #
    classifier = MLPRegressor((64,64,64), activation = "relu", solver='adam', max_iter=1000)

    # Train the classifier on the training data
    classifier.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = classifier.predict(X_test).clip(0, 5).round()

    # Evaluate the performance of the classifier
    accuracy = metrics.accuracy_score(y_test, y_pred)

    # Print the confusion matrix
    conf_matrix = metrics.confusion_matrix(y_test, y_pred)

    # Print the classification report
    class_report = metrics.classification_report(y_test, y_pred)

    return classifier, accuracy, conf_matrix, class_report

In [51]:
# train_multinomial_logistic_regression(train, "price")

In [55]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

def train_random_forest(data, test, target_column, n_folds=5, test_size=0.2, random_state=19):
    """
    Trains a Random Forest classifier and evaluates its performance.

    Parameters:
    - X: Features (independent variables)
    - y: Target variable (dependent variable)
    - n_estimators: The number of trees in the forest (default is 100)
    - test_size: Proportion of the dataset to include in the test split (default is 0.2)
    - random_state: Seed used by the random number generator (default is 42)

    Returns:
    - classifier: Trained Random Forest classifier
    - accuracy: Accuracy of the classifier on the test set
    - conf_matrix: Confusion matrix
    - class_report: Classification report
    """


    X = data.drop([target_column, 'id'], axis=1)
    y = data[target_column]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Create a Random Forest classifier
    classifier = RandomForestClassifier(n_estimators=100, random_state=random_state)

    param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
    # ,
    # 'max_features': ['auto', 'sqrt', 'log2', None],
    # 'bootstrap': [True, False]
    }
    # param_grid = {}

    stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)

    # Create GridSearchCV
    grid_search = GridSearchCV(classifier, param_grid, cv=stratified_kfold, scoring='accuracy')

    # Fit the model
    grid_search.fit(X_train, y_train)

    # Get the best estimator
    best_classifier = grid_search.best_estimator_

    # Make predictions on the test set
    y_pred = best_classifier.predict(X_test)

    # Evaluate the performance of the best classifier
    accuracy = metrics.accuracy_score(y_test, y_pred)

    # Make predictions on the final test set
    test = test.drop("id", axis=1)
    y_final = best_classifier.predict(test)

    return accuracy, y_pred, y_final, best_classifier

accuracy, y_pred, y_final, best_classifier = train_random_forest(train, test, "price")
print(f"Accuracy: {accuracy:.2f}")


Accuracy: 0.56


In [56]:
# Create a DataFrame with IDs and Predictions
output_df = pd.DataFrame({'id': ids.astype(int), 'price': y_final})

# Save the DataFrame to a CSV file
output_df.to_csv('predictions_rf.csv', index=False)